In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip install InstructorEmbedding faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 50.3 MB/s eta 0:00:00


In [ ]:
# !pip -q install langchain openai tiktoken chromadb pypdf sentence_transformers InstructorEmbedding faiss-cpu youtube-transcript-api

In [ ]:
import os
from dotenv import find_dotenv,load_dotenv

from langchain import HuggingFaceHub
from langchain import PromptTemplate, LLMChain
from langchain.document_loaders import YoutubeLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

import textwrap

In [ ]:
# Formating the output looks better
def format_text(input:str):
    wrapped_text = textwrap.fill(text=input,break_long_words=False, replace_whitespace=False,width=100)
    return wrapped_text

In [ ]:
# Setup the model
load_dotenv(find_dotenv())
HUGGINGFACEHUB_API_TOKEN = os.environ['HUGGINGFACEHUB_API_TOKEN']

In [ ]:
# loading the model from api
repo_id = 'tiiuae/falcon-7b-instruct'
# repo_id = 'tiiuae/falcon-40b'

falcon_llm = HuggingFaceHub(
    repo_id=repo_id,model_kwargs={"temperature":0.2,"max_new_tokens":500}
    )

In [ ]:
# # Testing the model
# template = """
# Question: {question}

# Answer:
# """

# question = "Who won the FIFA World Cup in every version "

# prompt = PromptTemplate(template=template, input_variables=['question'])
# llm_chain = LLMChain(prompt=prompt,llm=falcon_llm)

# response = llm_chain.run(question)
# print(format_text(response))

In [ ]:
# Youtube
# video_url = "https://www.youtube.com/watch?v=Jcoam1CeAq4"
# video_url = "https://youtu.be/U-BHz_UIOfs"
video_url = "https://youtu.be/qEk_1QdKuz4"
loader = YoutubeLoader.from_youtube_url(video_url)
transcript = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000)
docs = text_splitter.split_documents(transcript)

In [ ]:
# Summurize text
summerize_chain = load_summarize_chain(falcon_llm,chain_type="map_reduce",verbose=True)
# print(summerize_chain.llm_chain.prompt.template)
# print(summerize_chain.combine_document_chain.llm_chain.prompt.template)

output_summary = summerize_chain.run(docs)
print(format_text(output_summary))



> Entering new  chain...


> Entering new  chain...
Prompt after formatting:
Write a concise summary of the following:


"hello everybody there is construction going on right near where I live on two separate spaces so it's going to be very loud okay it's gonna be a bit but I got a new microphone okay oh well I always had this microphone but I I never trusted it but turns out it kind of works uh so I'm gonna use it today I'm going to be talking about what is meant for you we'll find you and this is one of the topics that our most near and dear to my heart because as humans we chase a lot of things in this life and chasing things is almost like the goal that we prescribe for each and every one of us you see that a lot in Hustle culture you see that in every kind of work culture around the world you see the sense of chasing your dreams chasing what you want while to some extent there is truth to that chasing your dreams I don't think that it's the truth in its entirety I think you need



> Entering new  chain...
Prompt after formatting:
Write a concise summary of the following:


"
The video discusses the importance of chasing one's dreams and finding what is meant for them. It highlights the concept of livelihood and how it is related to finding one's purpose in life.


The word 'word' encapsulates a concept that is meant for you, and it will find its way to you. It is not necessary to stress about it, as it will come to you naturally. Similarly, Muslims have a specific purpose in life, and it will find its way to them.


The author reflects on their journey of learning about their faith and how it has helped them navigate challenges in their personal and professional life. They emphasize the importance of trusting Allah and embracing uncertainty, as well as the significance of finding one's identity and purpose.


The author believes that people should trust Allah and not force things to happen, as trying to force a certain lifestyle often leads to negative outcome

In [ ]:
" ".join( """
Global migration is a critical issue with 280 million international migrants contributing to
economic growth and innovation. Companies and governments can benefit from migrants' unique
insights, experiences, and perspectives. Talent and opportunity are often hindered by visa policies,
bureaucracy, and language barriers. Companies can overcome these challenges by prioritizing hiring
globally and adapting to changing business needs. The pandemic has forced companies to adapt to
remote work, and this has led to a shift in hiring practices. Companies are now hiring globally and
looking for diverse talent pools. In addition, they are also hiring more globally and hiring
developers and for other roles internationally. It is important to be aware of biases and to check
them at the door.
""".split())

"Global migration is a critical issue with 280 million international migrants contributing to economic growth and innovation. Companies and governments can benefit from migrants' unique insights, experiences, and perspectives. Talent and opportunity are often hindered by visa policies, bureaucracy, and language barriers. Companies can overcome these challenges by prioritizing hiring globally and adapting to changing business needs. The pandemic has forced companies to adapt to remote work, and this has led to a shift in hiring practices. Companies are now hiring globally and looking for diverse talent pools. In addition, they are also hiring more globally and hiring developers and for other roles internationally. It is important to be aware of biases and to check them at the door."

In [ ]:
model_name = "hkunlp/instructor-xl"
model_kwargs = {"device": "cuda"}
embeddings =  HuggingFaceInstructEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

load INSTRUCTOR_Transformer
max_seq_length  512


In [ ]:
def create_db_from_yt_video(video_url):
    loader = YoutubeLoader.from_youtube_url(video_url)
    transcript = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap= 100)
    docs = text_splitter.split_documents(transcript)

    docs_db = FAISS.from_documents(docs,embeddings)
    return docs_db

In [ ]:
docs_db = create_db_from_yt_video(video_url)

In [ ]:
docs_db.similarity_search_with_score('Health')[-1]

(Document(page_content="where especially David Goggins talks about how how he struggled through his life and he would take anecdotes from the book and he would kind of describe it in more detail but the book itself I just cannot get enough of it I'm like obsessed with the book like if you're genuinely serious about changing your life you have got to read can't hurt me like I this is one of next I was gonna do this next of colon it's not it's literally not stuff oh my God but this is like it's like the Quran and like the books of the Sunnah and then underneath them comes can't hurt me but but like this is how much I trust in this book you know and it's not a book where he talks about what you should do in your life it's literally just a book where he talks about what he went through what his life was about what he had to endure as a child and I mean this guy went through so much but the thing that I could take from this book as much as I could with no hesitation and with no questions as

In [ ]:
retriever = docs_db.as_retriever(kwargs={'k':3})

In [ ]:
retriever.search_type

'similarity'

In [ ]:
retriever.get_relevant_documents('the ending')[-1].page_content

"know it's basically it's the gist of what is meant for you you know and within that meaning of the word like you know what that word encapsulates at the same time is that what is meant for you will find you like it will make its way to you you don't have to stress about that and like every time we see something that we like that other Muslims have we say you know have others go home that's what they are meant to have and I I don't know I just find that like such a beautiful concept it's not like something that they just have it was just meant for them all along you know it was written for them from before they were born so yeah I saw these birds and it just blossomed this idea in me of like what about humans you know if this is what birds have if this Birds go out every day and they find something like that what about humans I mean it has to be like a much grander scheme and honest to God when I see my life you know when I look back at the things that I've experienced the idea of what

First way - using RetrievalQA

In [ ]:
# falcon_llm
def qa_retrieval_chain(docs_db,llm,k=3):
  retriever = docs_db.as_retriever(kwargs={'k':k})

  chain =  RetrievalQA.from_chain_type(
                  llm=llm,
                  chain_type="stuff",
                  retriever=retriever,
                  return_source_documents=True,
                                          )

  return chain

Second way - uisng similarty search

In [ ]:
def qa_simlarity_search_chain(llm):
  template = """
        You are a helpful assistant that that can answer questions about youtube videos
        based on the video's transcript: {docs}

        Only use the factual information from the transcript to answer the question.

        If you feel like you don't have enough information to answer the question, say "I don't know".

        Your answers should be verbose and detailed.
        """
  system_message_template = SystemMessagePromptTemplate.from_template(template)

  human_template = "Answer the following question: {question}"
  human_message_template = HumanMessagePromptTemplate.from_template(human_template)

  chat_prompt = ChatPromptTemplate.from_messages(
      [system_message_template,human_message_template]
  )

  return LLMChain(llm=llm,prompt=chat_prompt)

def get_response_from_query_sim(chain,query,docs_db,k):
  docs = docs_db.similarity_search(query,k=k)
  docs_pc = " ".join([doc.page_content for doc in docs])

  response = chain.run(question=query,docs=docs_pc)

  return response, docs

In [ ]:
sim_chain = qa_simlarity_search_chain(falcon_llm)
ret_chain = qa_retrieval_chain(docs_db,falcon_llm,k=4)

In [ ]:
query = 'What is the name of the book mentioned in the video?'

print('-------------------Similarity------------------')
llm_response = get_response_from_query_sim(sim_chain,query,docs_db,3)
print(format_text(llm_response[0]))


print('-------------------Retrieval------------------')
llm_response = ret_chain(query)
print(format_text(llm_response['result']))

-------------------Similarity------------------


The name of the book mentioned in the video is "Can't Hurt Me" by David Goggins.
-------------------Retrieval------------------

David Goggins - Can't Hurt Me


In [ ]:
query = 'what does the book talk about (Cant Hurt Me) based on the video?'

print('-------------------Similarity------------------')
llm_response = get_response_from_query_sim(sim_chain,query,docs_db,3)
print(format_text(llm_response[0]))


print('-------------------Retrieval------------------')
llm_response = ret_chain(query)
print(format_text(llm_response['result']))

-------------------Similarity------------------


The book "Can't Hurt Me" by David Goggins talks about overcoming obstacles and challenges in life.
It is a book that can help anyone who wants to make a change in their life and overcome their
struggles. The book is about how to overcome obstacles and challenges in life and how to make a
change. It is a book that can help anyone who wants to make a change in their life and overcome
their struggles.
-------------------Retrieval------------------


The book "Can't Hurt Me" by David Goggins talks about overcoming obstacles and challenges in life.
It's not just about achieving your goals, but also about learning to overcome obstacles and
challenges that come your way. The book is a great read and can be very inspiring for anyone looking
to make a change in their life.


In [ ]:
query = 'based on the video What is the benefit of immigrants?'

print('-------------------Similarity------------------')
llm_response = get_response_from_query_sim(sim_chain,query,docs_db,3)
print(format_text(llm_response[0]))


print('-------------------Retrieval------------------')
llm_response = ret_chain(query)
print(format_text(llm_response['result']))

-------------------Similarity------------------


I think one of the biggest benefits of immigrants is that they bring a new set of skills and
knowledge to the country they are migrating to. They can help create new businesses, new jobs, and
new industries. They can also bring a new perspective to the country they are migrating to, and can
help to create a more diverse and vibrant society. Additionally, they can help to fill gaps in the
labor market, and can help to create a more competitive economy. Finally, they can also help to
create a more welcoming and inclusive society.
-------------------Retrieval------------------
 Immigrants bring new skills, ideas, and perspectives to companies, which can help them innovate and
grow. They also contribute to the local economy and can help alleviate labor shortages.


In [ ]:
query = "what is the name of Ali's business coach?"

print('-------------------Similarity------------------')
llm_response = get_response_from_query_sim(sim_chain,query,docs_db,3)
print(format_text(llm_response[0]))


print('\n-------------------Retrieval------------------')
llm_response = ret_chain(query)
print(format_text(llm_response['result']))

In [ ]:
query = "who is Eric?"

print('-------------------Similarity------------------')
llm_response = get_response_from_query_sim(sim_chain,query,docs_db,4)
print(format_text(llm_response[0]))


print('\n-------------------Retrieval------------------')
llm_response = ret_chain(query)
print(format_text(llm_response['result']))

In [ ]:
query = "Does the video recommend watching star wars film?"

print('-------------------Similarity------------------')
llm_response = get_response_from_query_sim(sim_chain,query,docs_db,2)
print(format_text(llm_response[0]))


print('\n-------------------Retrieval------------------')
llm_response = ret_chain(query)
print(format_text(llm_response['result']))

Video Summurize

In [ ]:
# Youtube
video_url = "https://www.youtube.com/watch?v=Jcoam1CeAq4"
loader = YoutubeLoader.from_youtube_url(video_url)
transcript = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000,chunk_overlap=200)
docs = text_splitter.split_documents(transcript)

In [ ]:
summarize_chain = load_summarize_chain(llm=falcon_llm, chain_type='map_reduce',verbose=True)

In [ ]:
output_summary = summarize_chain.run(docs)
text = format_text(output_summary)
print('-------------------Your summary------------------\n')
print(text)